In [2]:
import geopy as geopy
import requests
import selenium as selenium
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def handle_location_geo(url):
    driver = None
    try:
        # Set Chrome options to run headless and add custom headers
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")

        # Create WebDriver instance with custom options
        driver = webdriver.Chrome(options=chrome_options)

        # Load the URL in the driver
        driver.get(url)

        # Wait for the button to be present in the DOM
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='root_element']/main/div[2]/div/div/button[2]")))

        # Click the button
        button.click()

        # Wait for the new page to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="portal-root"]/div/div/div/div/div/div[3]/div[4]/div/div/a[3]')))

        # Now you can parse the updated page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        de = soup.select(
            '#portal-root > div > div > div > div > div > div.mapboxgl-control-container > div.mapboxgl-ctrl-bottom-right > div > div > a.mapbox-improve-map')
        print("de#########################################", de[0]['href'])

        # Split the URL at the '#' character and take the second part
        coords_part = de[0]['href'].split('#/')[1]

        # Split the coordinates part at the '/' character and take the first two parts
        latitude, longitude = coords_part.split('/')[:2]

        print("Latitude:", latitude)
        print("Longitude:", longitude)
        return latitude, longitude

    except Exception as e:
        print("Error:", e)
        return None, None

    finally:
        if driver is not None:
            driver.quit()


def page_handle(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    page = requests.get(url, headers=headers)
    src = page.content
    soup = BeautifulSoup(src, "html.parser")
    details = soup.find_all("div", {"class": "styles_desktop_list__item__lF_Fh"})
    if len(details) > 2:
        details = soup.find_all("div", {"class": "styles_desktop_list__item__lF_Fh"})
        descreption = soup.find_all("article", {"data-testid": True})
        location = soup.find_all("div", {"class": "styles_desktop_subtitle__container__msqhF"})
        development = soup.find_all("div", {"class": ["styles_desktop_broker__name__container__Hsrhs"]})
        price = soup.find("div", {"class": ["styles_desktop_price__row__kV3nj"]})
        pay_det = soup.find_all("div", {"class": ["styles_desktop_down-payment__box__D6mcQ"]})
        some_detail=soup.find("div", {"class": ["styles_attributes__list__sTe9h"]})
        some_detail_text=some_detail.text if some_detail else "unknown"
        # Find the <img> tag inside the URL
        # Find the button
        description_text = descreption[0].text if descreption else "unknown"
        location_text = location[0].text if location else "unknown"
        development_text = development[0].text if development else "unknown"
        print("desc", description_text)
        print("loc", location_text)
        print("dev", development_text)
        print("price", price.text)
        det=[]
        hd=get_house_details(details)
        return hd, description_text, location_text, development_text, price.text, pay_det
    else:
      print("not ok")
      details = soup.find_all("div", {"class": "styles_desktop_list__item__lF_Fh"})
      descreption = soup.find_all("article", {"data-testid": True})
      location = soup.find_all("div", {"class": "styles_desktop_subtitle__container__msqhF"})
      development = soup.find_all("div", {"class": ["styles_desktop_broker__name__container__Hsrhs"]})
      price = soup.find("div", {"class": ["styles_desktop_price__m_hT3 styles_attributes__price-container__jlc6e"]})
      pay_det = soup.find_all("div", {"class": ["styles_desktop_down-payment__box__D6mcQ"]})
      some_detail=soup.find("div", {"class": ["styles_attributes__list__sTe9h"]})

      description_text = descreption[0].text if descreption else "unknown"
      location_text = location[0].text if location else "unknown"
      development_text = development[0].text if development else "unknown"
      price_text = price.text if price else "unknown"
      some_detail_text=some_detail.text if some_detail else "unknown"
      print("some_detail_text",some_detail_text)
      print("desc", description_text)
      print("loc", location_text)
      print("dev", development_text)
      print("price", price_text)
      det=[]
      det.append(details[0].text)

      # Use regular expressions to extract the details
      pattern = r'(\d+) Bedrooms( \+ Maid)?(\d+) Bathrooms ([\d,]+) sqft / ([\d,]+) sqm'
      match = re.search(pattern, some_detail_text)

      if match:
          bedrooms, maid, bathrooms, sqft, sqm = match.groups()

          # If '+ Maid' is present, increment the bedroom count
          if maid:
              bedrooms = str(int(bedrooms) + 1)

          # Format the details in the desired format
          det.append(f"Bedrooms: {bedrooms}")
          det.append(f"Bathrooms: {bathrooms}")
          det.append(f"Property Size: {sqft} sqft / {sqm} sqm")
      house_dt=get_house_details_not(det)
      return house_dt, description_text, location_text, development_text, price_text, pay_det


def get_house_details(details):
    house_details = {}
    for i in range(len(details)):
        all_text = details[i].text.split(':')
        key = all_text[0]
        value = all_text[1]
        house_details[key] = value

    return house_details
def get_house_details_not(details):
    house_details = {}
    for i in range(len(details)):
        all_text = details[i].split(':')
        key = all_text[0]
        value = all_text[1]
        house_details[key] = value

    return house_details

def get_house_payment_details(details):
    payment_details = {}
    for i in range(len(details)):
        if 'Payment method' in details[i].text:
            value = details[i].text.split('Payment method')[1]
            payment_details['Payment method'] = value
        else:
            value = details[i].text.split('Down payment')[1]
            payment_details['Down payment'] = value
    return payment_details


def handle_location_name(location):
    loc = location.split(',')
    compound = loc[0]
    city = loc[1]
    return compound, city


def get_all_pages(main_page):
    src = main_page.content
    soup = BeautifulSoup(src, "html.parser")
    # Find all <a> tags with a title attribute
    links = soup.find_all('ul', {"class": "styles_desktop_container__V85pq"})
    # Use a set to store unique URLs
    unique_urls = set()

    # Loop through the found links and extract URLs
    for link in links[0].find_all_next('li'):

        url = link.find_next('a')['href']
        # Check if the URL ends with .html
        if url.endswith(".html") and url.startswith("https"):
            unique_urls.add(url)
    return list(unique_urls)


details = []
latl = []
longl = []
descl = []
devl = []
locl = []
all_urls = []
payment = []
pricel = []
for i in range(2550, 2600):
    print(i)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    main_page = requests.get(f"https://www.propertyfinder.eg/en/search?c=1&fu=0&ob=mr&page={i}", headers=headers)
    print(main_page)

    unique_urls = get_all_pages(main_page)
    print(len(unique_urls))
    count = 0
    for u in unique_urls:
        print(u)
        count += 1
        hd, des, loc, dev, price, pay_det = page_handle(u)
        lat, long1 = handle_location_geo(u)
        pd = get_house_payment_details(pay_det)
        print("hd##########", hd)
        print("pd#########", pd)
        details.append(hd)
        latl.append(lat)
        longl.append(long1)
        descl.append(des)
        devl.append(dev)
        locl.append(loc)
        all_urls.append(u)
        payment.append(pd)
        pricel.append(price)

2550
<Response [200]>
25
https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-south-investors-area-90-avenue-5279236.html
not ok
some_detail_text 2 Bedrooms2 Bathrooms 1,776 sqft / 165 sqm
desc Apartment for Sale in 90 Avenue Compound, Fifth Settlement90th Street in front of the American University directly (AUC)The apartment is fully finished ultra-modernBUA: 165 sqm (2 bedrooms + 2 bathrooms) with a direct view of the poolCash down payment: (50%)and the rest over 3 years without interest.For details: 01062525558Or via WhatsApp: [link not available]------------------------------------------------------------------About Tabarak Real Estate Development:---------------------------------------------For more than 37 years, we have been working with dedication and loyalty. We have successfully transitioned from a single business unit to a holding company.Throughout our journey of work and success, the group has embodied excellence in all its endeavors, where cred

KeyboardInterrupt: 

In [ ]:
import csv
import itertools
# Extract all keys from all dictionaries
all_keys_details = set().union(*(d.keys() for d in details))
all_keys_payment = set().union(*(p.keys() for p in payment))

# Combine all keys from details and payment
all_keys = all_keys_details.union(all_keys_payment)

# Define the CSV file name
csv_file = "locations_real_c1_t_2550to_2597.csv"

# Specify encoding for Arabic text
encoding = 'utf-8-sig'

# Convert all_keys to a list and concatenate it with the additional column names
fieldnames = list(all_keys) + ['compound', 'dev', 'latitude', 'longitude', 'description', 'url', 'price']

# Initialize counter
count = 0

# Write the data to the CSV file
with open(csv_file, mode='w', newline='', encoding=encoding) as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data
    for lat, long1, desc, u, house_details, comp,dev, price, pay in itertools.zip_longest(latl, longl, descl, all_urls, details, locl, devl, pricel, payment):
        try:
            # Fill in missing keys with None
            if house_details is None:
                house_details = {key: None for key in all_keys}
            if pay is None:
                pay = {key: None for key in all_keys}

            row = {**house_details, **pay, 'compound': comp, 'dev': dev, 'latitude': lat, 'longitude': long1, 'description': desc, 'url': u, 'price': price}

            # Write the row to the CSV filewww
            writer.writerow(row)

            # Increment counter
            count += 1
        except Exception as e:
            print(f"Error writing row: {e}")

# Print the total number of rows written
print(f"Total rows written: {count}")

# Print the name of the CSV file where data has been saved
print("Data has been saved to", csv_file)

Total rows written: 1175
Data has been saved to locations_real_c1_t_2550to_2597.csv
